## Snap POIs to Road Network

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pickle
import osmnx as ox

import sys

sys.path.append("/app")
from constants import DATA_PATH

import os

from db_utils import get_engine, get_table_creation_query

import logging

import dataloader as loader
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
from network_utils import get_closest_osmids


<IPython.core.display.Javascript object>

## Connect to Database

In [3]:
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")

<IPython.core.display.Javascript object>

In [4]:
engine = get_engine(user, password, host)

<IPython.core.display.Javascript object>

In [5]:
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter("%(asctime)s [%(name)s] %(levelname)-8s %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

In [6]:
pois = loader.load_poi_gdfs(user=user, password=password, host=host)

<IPython.core.display.Javascript object>

In [7]:
is_point = {
    category: np.all(pois[category].geometry.geom_type == "Point") for category in pois
}

<IPython.core.display.Javascript object>

In [8]:
poi_coords = {
    category: np.array(
        [[p.geometry.x, p.geometry.y] for p in pois[category].itertuples()]
    )
    for category in pois
    if is_point[category]
}

<IPython.core.display.Javascript object>

In [9]:
with engine.connect() as conn:
    df_nodes = pd.read_sql("SELECT * FROM node_coords", con=conn)

<IPython.core.display.Javascript object>

In [10]:
coords_nodes = df_nodes[["eastings", "northings"]].values

<IPython.core.display.Javascript object>

In [11]:
closest_osmids = {
    category: get_closest_osmids(
        np.array([[p.geometry.x, p.geometry.y] for p in pois[category].itertuples()]),
        coords_nodes,
        df_nodes.osmid.values,
    )
    for category in pois
    if is_point[category]
}

<IPython.core.display.Javascript object>

In [12]:
output_dfs_points = {
    category: pd.DataFrame(
        {
            "poi_id": pois[category].id.values,
            "osmid": closest_osmids[category],
            "poi_category": category,
        }
    )
    for category in closest_osmids
}

<IPython.core.display.Javascript object>

In [13]:
output = pd.concat([output_dfs_points[i] for i in output_dfs_points])


<IPython.core.display.Javascript object>

In [14]:
cols = {
    "poi_id": "INTEGER",
    "osmid": "BIGINT",
    "poi_category": "VARCHAR(64)",
}

index_cols = ["poi_id", "osmid", "poi_category"]
unique_cols = []

<IPython.core.display.Javascript object>

In [15]:
create_q = get_table_creation_query(
    "poi_nodes", cols, "public", index_cols, unique_cols
)

<IPython.core.display.Javascript object>

In [16]:
with engine.connect() as conn:
    conn.execute(create_q)

<IPython.core.display.Javascript object>

In [17]:
with engine.connect() as conn:
    output.to_sql("poi_nodes", index=False, con=conn, if_exists="append")

<IPython.core.display.Javascript object>